# Infleqtion DCM4 Quantum Challenge

#### Air Defence Quantum Computing Use-case

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Infleqtion/client-superstaq/blob/dcm4/Infleqtion-DCM4-Challenge/Infleqtion_DCM4_Challenge.ipynb)

## Problem Description

Consider a set of $T$ offensive targets engaged in an offensive operation against $K$ defensive
assets. We assume that through intelligence operations the defensive entity know that the set of
offensive targets that threaten asset $k$ is $T_k$.

Suppose each defensive asset has a strategic value $v_k$ and that the probability of offensive
target $j$ destroying the asset $k$ is $p_{j, k}$.

The defensive entity has $W$ weapon systems with which to defend its assets. Assume the probability
that weapon system $i$ destroys the offensive target $j$ is $q_{i, j}$.


![link](images/Problem_Description.png)

## <b><font color='green'>Goal: Optimise air defence to maximise asset survival.</font></b>
In this notebook, we will explore an application of quantum computing to the field of logistics optimisation. You are given a **quantum algorithm** for solving this problem. **Your objective is to select appropriate parameter values to maximise asset survival.**

## <b><font color="Red">Wherever you should insert code, you will see the following code block:</font></b>
```
# ##############################################
#
# **INSERT CODE HERE**
#
#  ##############################################
```
E.g., to insert the value `10` for a variable named `PARAMETER`, replace
```
PARAMETER = # **INSERT CODE HERE**
```
with the following:
```
PARAMETER = 10
```

### <b><font color="Red">Please do not modify any other existing code!</font></b>
**Don't forget to execute every cell in sequence.**

In [ ]:
requirements = """
cirq-superstaq~=0.5.31
dimod~=0.12.18
scipy~=1.14.1
tabulate~=0.9.0
"""
with open('dcm4-challenge-requirements.txt', 'w') as f:
    f.write(requirements)

TOOLS_URL = "https://raw.githubusercontent.com/Infleqtion/client-superstaq/refs/heads/dcm4/Infleqtion-DCM4-Challenge/tools.py"
!wget {TOOLS_URL} -O ./tools.py --quiet

try:
    import tools
except ImportError:
    print("Installing requirements...")
    %pip install --quiet -r dcm4-challenge-requirements.txt
    print("Installed requirements.")
    print("You may need to restart the kernel to import newly installed packages. See the `Kernel` dropdown menu in the toolbar.")
    import tools

import time
import numpy as np
from tabulate import tabulate

## Define Problem
We can now define an instance of the above problem and represent the solution graphically.

In [ ]:
np.random.seed(250297)

# Number of entities
num_targets = 4
num_weapons = 2
num_assets = 3

# Indexes
T = np.arange(num_targets)
W = np.arange(num_weapons)
K = np.arange(num_assets)

# Values
asset_values = np.array([1.5, 3.6, 8.3])

# Engagement probabilities
asset_destruction_probabilities = np.array(
    [
        [0.1, 0.07, 0.14],
        [0.03, 0.06, 0.12],
        [0.2, 0.3, 0.29],
        [0.12, 0.22, 0.05],
    ]
)
target_destruction_probabilities = np.array(
    [
        [0.3, 0.01, 0.15, 0.08],
        [0.05, 0.15, 0.4, 0.02],
    ]
)

# Constraint
weapon_ammunition_limits = np.array([2, 2])

# Build model
model = tools.AirDefenceModel(
    asset_values,
    asset_destruction_probabilities,
    target_destruction_probabilities,
    weapon_ammunition_limits,
)
# Plot solution
model.plot_solution(model.exact_solution)

## QUBO/Ising Formulation
Mathematical model representing the optimisation problem:

In [ ]:
print(f"J = \n{tabulate(model.J_matrix, tablefmt='heavy_grid')}")
print(f"h = \n{tabulate([model.h_vec], tablefmt='heavy_grid')}")
print(f"c = \n{tabulate([[model.offset]], tablefmt='heavy_grid')}")

## QAOA: Quantum Approximate Optimisation Algorithm

The Ising formulation is well suited to solving on a quantum computer as it can be associated with a
Hamiltonian that can be easily studied on a quantum computer. A Hamiltonian describes how a quantum
system behaves and importantly, the optimal solution of our optimisation problem maps to the ground
state of the associated Hamiltonian. The ground state is how the quantum system will arrange itself
under idealised conditions.

**QAOA** is an variational algorithm form finding the ground state by repeatedly "mixing" the quantum
state and then letting it settle again. We use a classical optimiser to vary length of time spent
mixing and settling in order to find the ground state of the system and hence our optimal solution.
Generally we will have far fewer classical parameters to optimise over than in the original problem.

### <b><font color="red">Your objective is to (a) maximise value of surviving assets and (b) minimise algorithm runtime.</font></b>

### <b><font color="red">Your final score will depend on both factors.</font></b>

Input values for the following:
- `NUMBER_OF_CIRCUIT_LAYER_REPETITONS`: The number of quantum circuit layers. In the theoretical limit, using longer quantum circuits in QAOA will increase success probability, but in practice this is at the expense of runtime and, on noisy quantum computers, increased frequency of errors. E.g., see `model.build_circuit(num_reps=NUMBER_OF_CIRCUIT_LAYER_REPETITIONS)` in the cells below.
- `NUMBER_OF_CIRCUIT_EXECUTION_SHOTS`: The number of times each circuit is executed, per iteration of the outer classical optimisation loop.
- `CLASSICAL_OPTIMISER_METHOD`: Type of classical optimiser used to modify quantum circuit parameters.
- `CONVERGENCE_TOLERANCE`: Represents the convergence criterion in the optimization; when the stepwise improvement becomes smaller than this tolerance, the optimization stops.
- `MAXIMUM_NUMBER_OF_ITERATIONS`: Maximum number of iterations to perform. Depending on the classical optimiser method, each iteration may use several function evaluations.

See https://docs.scipy.org/doc/scipy/reference/optimize.html#local-multivariate-optimization for `CLASSICAL_OPTIMISER_METHOD` options:
- `"Nelder-Mead"`
- `"Powell"`
- `"CG"`
- `"BFGS"`
- `"Newton-CG"`
- `"L-BFGS-B"`
- `"TNC"`
- `"COBYLA"`
- `"COBYQA"`
- `"SLSQP"`
- `"trust-constr"`
- `"dogleg"`
- `"trust-ncg"`
- `"trust-exact"`
- `"trust-krylov"`

In [ ]:
# ###############################################################
#
NUMBER_OF_CIRCUIT_LAYER_REPETITIONS = # **INSERT CODE HERE**
NUMBER_OF_CIRCUIT_EXECUTION_SHOTS = # **INSERT CODE HERE**
CLASSICAL_OPTIMISER_METHOD = # **INSERT CODE HERE**
CONVERGENCE_TOLERANCE = # ** INSERT CODE HERE  # Hint: this should be smaller than 1.
MAXIMUM_NUMBER_OF_ITERATIONS = # ** INSERT CODE HERE
#
# ###############################################################

E.g., if we set `NUMBER_OF_CIRCUIT_LAYER_REPETITIONS = N`, we have that `N` repetitions of this mixing and settling cycle looks like the quantum circuit below. The $\gamma$ and $\beta$ parameters are what we will optimise classically.

In [ ]:
model.build_circuit(num_reps=NUMBER_OF_CIRCUIT_LAYER_REPETITIONS)

Now we can use a classical optimizer to help us find the ground state, and hence the optimal
solution to our optimisation problem. <b>Please do not rename the variables `approx` and `runtime`!</b> These variables are used for calculating your final score.

In [ ]:
x0 = np.array(
    [
        1.16396594, 2.01322123, 1.02666917, -0.10892354,
        1.0941958, 0.86832268, 0.88237201, 0.95342334,
        0.96314088, 1.08413136, 1.07161153, 1.03514973,
        0.98488821, 1.07436944, 1.12914504, 0.98710917,
    ]
)
start = time.time()
res = model.solve_qaoa(
    x0=x0,
    nreps=NUMBER_OF_CIRCUIT_LAYER_REPETITIONS,
    shots=NUMBER_OF_CIRCUIT_EXECUTION_SHOTS,
    method=CLASSICAL_OPTIMISER_METHOD,
    tol=CONVERGENCE_TOLERANCE,
    maxiter=MAXIMUM_NUMBER_OF_ITERATIONS,
)
runtime = time.time() - start  # DO NOT RENAME THIS VARIABLE
approx = model.optimal_bitstring(res.x)  # DO NOT RENAME THIS VARIABLE
model.plot_solution(approx)
print(f"Runtime: {runtime:0.2f}s")
print("Final solution:", approx.flatten())

# <font color="green">**Congratulations! This concludes the DCM4 Quantum Challenge.**</font>

<b><font color="red">Please save this jupyter notebook with the following filename: `[Insert Your Team Name]_Infleqtion_DCM4_Challenge_Solution.ipynb`</font></b>

<b><font color="red">Email your final solution (completed jupyter notebook) to dcm@infleqtion.com</font></b>

# Appendix

### Combinatorial Optimisation Mathematical Model

Let $x_{i, j}$ be a binary variable that indicates whether weapon system $i$ is assigned to
intercept target $j$:

$$
x_{i, j} =
\begin{cases}
1& \text{Weapon $i$ assigned to intercept target }j\\
0& \text{otherwise}
\end{cases}
$$

We aim to maximise the expected value of all surviving assets following the engagement. The
objective function is given by $$c(x) = \sum_{k=1}^K v_k P(\text{Asset $k$ survives})$$ and we now
will see that $P(\text{Asset k survives})\propto x_{i,j}$.

$$
\begin{aligned}
    P(\text{Asset $k$ survives}) &= 1 - \sum_{j \in T_k} P(\text{$j$ attacks and destroys $k$})\\
    & = 1 - \sum_{j \in T_k} P(\text{$j$ survives the air defence}) P(\text{$j$ destroys $k$}) \\
    & = 1 - \sum_{j \in T_k} (1 - P(\text{$j$ destroyed by air defence}))P(\text{$j$ destroys $k$}) \\
    & = 1 - \sum_{j \in T_k} \left(1 - \sum_{i=1}^{W}P(\text{$i$ destroys $j$})\right)P(\text{$j$ destroys $k$}) \\
    & = 1 - \sum_{j \in T_k} \left(1 - \sum_{i=1}^{W}x_{i, j}q_{i, j}\right)p_{j, k}
\end{aligned}
$$

Giving
$$c(x) = \sum_{k=1}^K v_k \left(1 - \sum_{j \in T_k} \left(1 - \sum_{i=1}^{W}x_{i, j}p_{i, j}\right)q_{j, k}\right)$$
which is a linear objective function in the decision variables $x_{i, j}$.

Finally the weapon systems are limited in the volume of ammunition they store, and thus the number
of targets that can engage (assuming for simplicity each weapon system uses one unit of ammunition
to engage with each target and that a target cannot be engaged multiple times). We make the
observation that at optimality this constraint will be tight and hence
$$\sum_{j=1}^{T}x_{i, j} = N_i$$

where $N_i$ is the ammunition limit of weapon system $i$.

### QUBO / Ising Model Formulation

By adding Lagrange multipliers to handle the constraint terms, the above model can be written in the
from $$\min_{\vec{x} \in \mathbb{B}^n} \ \ \vec{x}^T Q \vec{x} + \delta$$ known as the QUBO
formulation.

Switching the binary variables from $\{0, 1\}$ to $\{-1, 1\}$ we can reformulate this as
$$\min_{\vec{y} \in \mathbb{S}^n} \ \ \vec{y}^T J \vec{y} + \vec{y}^T \vec{h} + c$$

for some matrix $J$, vector $\vec{h}$ and scalar $c$. As we will see below, this Ising formulation
is well suited to being solved by a quantum computer where the $y \in \{-1, 1\}$ are associated with
the $\pm 1$ eigenvalues of the $Z$ gate.